In [ ]:
from pyomo.environ import *


Model = ConcreteModel()

Model.n =  RangeSet(1,10)

Model.n0 = RangeSet(1,2)

Model.nc = RangeSet(3,10)






x1 = {1:117 , 2:79 , 3:92 , 4:20 ,5:83 , 6:69 , 7:42 , 8:94 , 9:74 , 10:69}

Model.x1 = Param(Model.n,initialize = x1)



y1 = {1:174 , 2:29 , 3:162 , 4:86 , 5:43 , 6:212 , 7 :50 , 8:118 , 9:64 , 10:194}

Model.y1 = Param(Model.n,initialize = y1)




d = {3:112 , 4:112 , 5:62 , 6:145 , 7:135 , 8:128 , 9:71 , 10:54}

Model.d = Param(Model.nc,initialize = d)



p = {3:23 , 4:200 , 5:13 , 6:200 , 7:27 , 8:200 , 9:15 , 10:98}

Model.p = Param(Model.nc,initialize = p)



fd ={1:36 , 2:33}

Model.fd = Param(Model.n0,initialize = fd)



cd = 1000


cv = 200

fv =20


c ={}
for i in Model.n:
    for j in Model.n:
               c[i,j] = ((x1[i]-x1[j])**2+ (y1[i]-y1[j])**2)**0.5

Model.c = Param(Model.n,Model.n ,initialize = c)



Model.x  = Var(Model.n , Model.n , within = Binary)


Model.y = Var(Model.n0  , within = Binary)

Model.z = Var(Model.nc , Model.n0  , within = Binary)


Model.v = Var(Model.n,Model.n , within = PositiveReals)

Model.u = Var(Model.n,Model.n , within = PositiveReals)




def _2 (Model,i):
     
        xij=0
        for j in Model.n:
            xij+=Model.x[i,j]
            return xij ==1
    
Model._2 = Constraint(Model.nc,rule =_2)



def _3 (Model,i):
    
        xij=0
        xji=0
        for j in Model.n:
            xij+=Model.x[i,j]
            xji+=Model.x[j,i]
        return xij == xji
Model._3 = Constraint(Model.n,rule =_3)



def _4 (Model,i):
    
        uij = 0
        uji = 0
        for j in Model.n:
            uji +=Model.u[j,i]
            uij +=Model.u[i,j]
        return uji - uij == Model.d[i]
Model._4 = Constraint(Model.nc,rule =_4)



def _5 (Model,i):
    
        vij = 0
        vji = 0
        for j in Model.n:
            vji +=Model.v[j,i]
            vij +=Model.v[i,j]
        return vji - vij == Model.p[i]
Model._5 = Constraint(Model.nc,rule =_5)


def _6 (Model,i,j):
  
            if i == j:
                Constraint.Skip
        
            return  Model.u[i,j] + Model.v[i,j] <=cv * Model.x[i,j]
            
                  
Model._6 = Constraint(Model.nc,Model.nc,rule =_6)



def _7 (Model,k):
    
        ukj = 0
        zjkdj = 0
        for j in Model.nc:
            ukj += Model.u[k,j]
            zjkdj +=Model.z[j,k]*Model.d[j]
        return ukj == zjkdj
Model._7 = Constraint(Model.n0,rule =_7)



def _8 (Model,k):
    
        ujk=0
        for j in Model.nc:
            ujk+=Model.u[j,k]
        return ujk == 0
Model._8 = Constraint(Model.n0,rule =_8)



def _9 (Model,k):
    
        vjk = 0
        zjkpj = 0
        for j in Model.nc:
            vjk += Model.v[k,j]
            zjkpj +=Model.z[j,k]*Model.p[j]
        return vjk == zjkpj
Model._9 = Constraint(Model.n0,rule =_9)



def _10 (Model,k):
    
        vkj=0
        for j in Model.nc:
            vkj+=Model.v[k,j]
        return vkj == 0
Model._10 = Constraint(Model.n0,rule =_10)




def _11 (Model,j,i):
   
            return Model.u[i,j] <= (cv - Model.d[i])* Model.x[i,j]
Model._11 = Constraint(Model.n,Model.nc,rule =_11)


def _12 (Model,i,j):
   
            return Model.v[i,j] <= (cv - Model.p[j])* Model.x[i,j]
Model._12 = Constraint(Model.n,Model.nc,rule =_12)



def _13 (Model,i,j):
   
            return Model.u[i,j] >= Model.d[j]*Model.x[i,j]
Model._13 = Constraint(Model.n,Model.nc,rule =_13)



def _14 (Model,j,i):
    
            return Model.v[i,j] >= Model.p[i] * Model.x[i,j]
Model._14 = Constraint(Model.n,Model.nc,rule =_14)

def _15 (Model,i):
    
        zik=0
        for k in Model.n0:
            zik += Model.z[i,k]
        return zik == 1
Model._15 = Constraint(Model.nc,rule =_15)

def _16 (Model,k):
    
        dizik = 0
        for i in Model.nc:
            dizik += Model.d[i] * Model.z[i,k]
        return dizik <=cd * Model.y[k]
Model._16 = Constraint(Model.n0,rule =_16)

def _17 (Model,k):
    
        pizik = 0
        for i in Model.nc:
            pizik += Model.p[i] * Model.z[i,k]
        return pizik <=cd * Model.y[k]
Model._17 = Constraint(Model.n0,rule =_17)

def _18 (Model,k):
    
        for i in Model.nc:
            return Model.x[i,k] <= Model.z[i,k]
Model._18 = Constraint(Model.n0,rule =_18)

def _19 (Model,k,i):
   
            return Model.x[k,i] <= Model.z[i,k]
Model._19 = Constraint(Model.n0,Model.nc,rule =_19)

def _20 (Model,i,j,k):
    
                if i==j :
                    Constraint.Skip
                zjm = 0
                for m in Model.n0:
                    if m == k:
                            Constraint.Skip
                    
                    zjm += Model.z[j,m]
                    return Model.x[i,j]+Model.z[i,k] + zjm <=2
Model._20 = Constraint(Model.nc,Model.nc,Model.n0,rule =_20)


def obj (Model):
    cx = 0
    fdy = 0
    fvx = 0
    for i in Model.n :
        for j in Model.n :
            cx += Model.c[i,j]*Model.x[i,j]
    for k in Model.n0 :
        fdy += Model.fd[k] * Model.y[k]
    for k in Model.n0 :
        for i in Model.nc :
            fvx += fv * Model.x[k,i]
    return cx + fdy + fvx
Model.objective = Objective(rule=obj)



def pyomo_postprocess(options=None, instance=None, results=None):
        Model.objective.display()
        

    #This is an optional code path that allows the script to be run outside of
    #pyomo command-line.  For example:  python transport.py
if __name__ == '__main__':
   # This emulates what the pyomo command-line tools does
    from pyomo.opt import SolverFactory
#   import pyomo.environ
    opt = SolverFactory("glpk")
    results = opt.solve(Model)
    #sends results to stdout
#    results.write()
#    print("\nDisplaying Solution\n" + '!'*60)
     pyomo_postprocess(None, Model, results)

In [75]:
Model.objective.display()

objective : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 873.5784213002651


In [39]:
x1


{1: 55, 2: 90, 3: 81, 4: 82, 5: 52, 6: 92, 7: 54, 8: 77, 9: 50, 10: 56}

In [40]:
y1

{1: 96, 2: 92, 3: 100, 4: 73, 5: 67, 6: 52, 7: 87, 8: 58, 9: 61, 10: 55}

In [41]:
fd

{1: 34, 2: 48}

In [42]:
d

{3: 95, 4: 56, 5: 83, 6: 61, 7: 62, 8: 90, 9: 58, 10: 92}

In [43]:
p

{3: 68, 4: 60, 5: 81, 6: 91, 7: 80, 8: 52, 9: 86, 10: 67}

In [44]:
cv

200

In [45]:
cd

500

In [46]:
fv

20

In [47]:
Model.n.display()

n : Dim=0, Dimen=1, Size=10, Domain=Integers, Ordered=True, Bounds=(1, 10)
    Virtual


In [48]:
Model.z.display()

z : Size=16, Index=z_index
    Key     : Lower : Value : Upper : Fixed : Stale : Domain
     (3, 1) :     0 :   1.0 :     1 : False : False : Binary
     (3, 2) :     0 :   0.0 :     1 : False : False : Binary
     (4, 1) :     0 :   1.0 :     1 : False : False : Binary
     (4, 2) :     0 :   0.0 :     1 : False : False : Binary
     (5, 1) :     0 :   0.0 :     1 : False : False : Binary
     (5, 2) :     0 :   1.0 :     1 : False : False : Binary
     (6, 1) :     0 :   1.0 :     1 : False : False : Binary
     (6, 2) :     0 :   0.0 :     1 : False : False : Binary
     (7, 1) :     0 :   1.0 :     1 : False : False : Binary
     (7, 2) :     0 :   0.0 :     1 : False : False : Binary
     (8, 1) :     0 :   0.0 :     1 : False : False : Binary
     (8, 2) :     0 :   1.0 :     1 : False : False : Binary
     (9, 1) :     0 :   0.0 :     1 : False : False : Binary
     (9, 2) :     0 :   1.0 :     1 : False : False : Binary
    (10, 1) :     0 :   1.0 :     1 : False : False : Bina

In [49]:
Model.x.display()

x : Size=100, Index=x_index
    Key      : Lower : Value : Upper : Fixed : Stale : Domain
      (1, 1) :     0 :   0.0 :     1 : False : False : Binary
      (1, 2) :     0 :   0.0 :     1 : False : False : Binary
      (1, 3) :     0 :   1.0 :     1 : False : False : Binary
      (1, 4) :     0 :   1.0 :     1 : False : False : Binary
      (1, 5) :     0 :   1.0 :     1 : False : False : Binary
      (1, 6) :     0 :   1.0 :     1 : False : False : Binary
      (1, 7) :     0 :   1.0 :     1 : False : False : Binary
      (1, 8) :     0 :   1.0 :     1 : False : False : Binary
      (1, 9) :     0 :   1.0 :     1 : False : False : Binary
     (1, 10) :     0 :   1.0 :     1 : False : False : Binary
      (2, 1) :     0 :   0.0 :     1 : False : False : Binary
      (2, 2) :     0 :  None :     1 : False :  True : Binary
      (2, 3) :     0 :   0.0 :     1 : False : False : Binary
      (2, 4) :     0 :   0.0 :     1 : False : False : Binary
      (2, 5) :     0 :   0.0 :     1 : Fal

In [50]:
Model.y.display()

y : Size=2, Index=y_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   1.0 :     1 : False : False : Binary
      2 :     0 :   0.0 :     1 : False : False : Binary


In [51]:
Model.v.display()

v : Size=100, Index=v_index
    Key      : Lower : Value : Upper : Fixed : Stale : Domain
      (1, 1) :     0 :  None :  None : False :  True : PositiveReals
      (1, 2) :     0 :   0.0 :  None : False : False : PositiveReals
      (1, 3) :     0 :  68.0 :  None : False : False : PositiveReals
      (1, 4) :     0 :  60.0 :  None : False : False : PositiveReals
      (1, 5) :     0 :  81.0 :  None : False : False : PositiveReals
      (1, 6) :     0 :  91.0 :  None : False : False : PositiveReals
      (1, 7) :     0 :  80.0 :  None : False : False : PositiveReals
      (1, 8) :     0 :  52.0 :  None : False : False : PositiveReals
      (1, 9) :     0 :  86.0 :  None : False : False : PositiveReals
     (1, 10) :     0 :  67.0 :  None : False : False : PositiveReals
      (2, 1) :     0 :   0.0 :  None : False : False : PositiveReals
      (2, 2) :     0 :  None :  None : False :  True : PositiveReals
      (2, 3) :     0 :   0.0 :  None : False : False : PositiveReals
      (2, 4) 

In [52]:
Model.u.display()

u : Size=100, Index=u_index
    Key      : Lower : Value : Upper : Fixed : Stale : Domain
      (1, 1) :     0 :  None :  None : False :  True : PositiveReals
      (1, 2) :     0 :   0.0 :  None : False : False : PositiveReals
      (1, 3) :     0 :  95.0 :  None : False : False : PositiveReals
      (1, 4) :     0 :   0.0 :  None : False : False : PositiveReals
      (1, 5) :     0 :  58.0 :  None : False : False : PositiveReals
      (1, 6) :     0 :  61.0 :  None : False : False : PositiveReals
      (1, 7) :     0 :  62.0 :  None : False : False : PositiveReals
      (1, 8) :     0 :  90.0 :  None : False : False : PositiveReals
      (1, 9) :     0 :   0.0 :  None : False : False : PositiveReals
     (1, 10) :     0 :   0.0 :  None : False : False : PositiveReals
      (2, 1) :     0 :   0.0 :  None : False : False : PositiveReals
      (2, 2) :     0 :  None :  None : False :  True : PositiveReals
      (2, 3) :     0 :   0.0 :  None : False : False : PositiveReals
      (2, 4) 

In [53]:
c

<function __main__.c>

In [54]:
Model._2.display()

_2 : Size=8
    Key : Lower : Body : Upper
      3 :   1.0 :  1.0 :   1.0
      4 :   1.0 :  1.0 :   1.0
      5 :   1.0 :  1.0 :   1.0
      6 :   1.0 :  1.0 :   1.0
      7 :   1.0 :  1.0 :   1.0
      8 :   1.0 :  1.0 :   1.0
      9 :   1.0 :  1.0 :   1.0
     10 :   1.0 :  1.0 :   1.0


In [55]:
Model._3.display()

_3 : Size=10
    Key : Lower : Body : Upper
      1 :   0.0 :  0.0 :   0.0
      2 :   0.0 :  0.0 :   0.0
      3 :   0.0 :  0.0 :   0.0
      4 :   0.0 :  0.0 :   0.0
      5 :   0.0 :  0.0 :   0.0
      6 :   0.0 :  0.0 :   0.0
      7 :   0.0 :  0.0 :   0.0
      8 :   0.0 :  0.0 :   0.0
      9 :   0.0 :  0.0 :   0.0
     10 :   0.0 :  0.0 :   0.0


In [56]:
Model._4.display()

_4 : Size=8
    Key : Lower : Body : Upper
      3 :  95.0 : 95.0 :  95.0
      4 :  56.0 : 56.0 :  56.0
      5 :  83.0 : 83.0 :  83.0
      6 :  61.0 : 61.0 :  61.0
      7 :  62.0 : 62.0 :  62.0
      8 :  90.0 : 90.0 :  90.0
      9 :  58.0 : 58.0 :  58.0
     10 :  92.0 : 92.0 :  92.0


In [57]:
Model._5.display()

_5 : Size=8
    Key : Lower : Body : Upper
      3 :  68.0 : 68.0 :  68.0
      4 :  60.0 : 60.0 :  60.0
      5 :  81.0 : 81.0 :  81.0
      6 :  91.0 : 91.0 :  91.0
      7 :  80.0 : 80.0 :  80.0
      8 :  52.0 : 52.0 :  52.0
      9 :  86.0 : 86.0 :  86.0
     10 :  67.0 : 67.0 :  67.0


In [58]:
Model._6.display()

_6 : Size=10
    Key : Lower : Body   : Upper
      1 :  None :    0.0 :   0.0
      2 :  None :    0.0 :   0.0
      3 :  None : -200.0 :   0.0
      4 :  None : -200.0 :   0.0
      5 :  None : -200.0 :   0.0
      6 :  None : -200.0 :   0.0
      7 :  None : -200.0 :   0.0
      8 :  None : -200.0 :   0.0
      9 :  None : -200.0 :   0.0
     10 :  None : -200.0 :   0.0


In [59]:
Model._7.display()

_7 : Size=2
    Key : Lower : Body : Upper
      1 :   0.0 :  0.0 :   0.0
      2 :   0.0 :  0.0 :   0.0


In [60]:
Model._8.display()

_8 : Size=2
    Key : Lower : Body : Upper
      1 :   0.0 :  0.0 :   0.0
      2 :   0.0 :  0.0 :   0.0


In [61]:
Model._9.display()

_9 : Size=1
    Key  : Lower : Body : Upper
    None :   0.0 :  0.0 :   0.0


In [62]:
Model._10.display()

_10 : Size=2
    Key : Lower : Body : Upper
      1 :   0.0 :  0.0 :   0.0
      2 :   0.0 :  0.0 :   0.0


In [63]:
Model._11.display()

_11 : Size=80
    Key      : Lower : Body   : Upper
      (3, 1) :  None : -105.0 :   0.0
      (3, 2) :  None :    0.0 :   0.0
      (3, 3) :  None :    0.0 :   0.0
      (3, 4) :  None :    0.0 :   0.0
      (3, 5) :  None :    0.0 :   0.0
      (3, 6) :  None :    0.0 :   0.0
      (3, 7) :  None :    0.0 :   0.0
      (3, 8) :  None :    0.0 :   0.0
      (3, 9) :  None :    0.0 :   0.0
     (3, 10) :  None :    0.0 :   0.0
      (4, 1) :  None : -144.0 :   0.0
      (4, 2) :  None :    0.0 :   0.0
      (4, 3) :  None :    0.0 :   0.0
      (4, 4) :  None :    0.0 :   0.0
      (4, 5) :  None :    0.0 :   0.0
      (4, 6) :  None :    0.0 :   0.0
      (4, 7) :  None :    0.0 :   0.0
      (4, 8) :  None :    0.0 :   0.0
      (4, 9) :  None :    0.0 :   0.0
     (4, 10) :  None :    0.0 :   0.0
      (5, 1) :  None : -117.0 :   0.0
      (5, 2) :  None :    0.0 :   0.0
      (5, 3) :  None :    0.0 :   0.0
      (5, 4) :  None :    0.0 :   0.0
      (5, 5) :  None :    0.0 :   0.

In [64]:
Model._12.display()

_12 : Size=80
    Key      : Lower : Body   : Upper
      (3, 1) :  None : -132.0 :   0.0
      (3, 2) :  None :    0.0 :   0.0
      (3, 3) :  None :    0.0 :   0.0
      (3, 4) :  None :    0.0 :   0.0
      (3, 5) :  None :    0.0 :   0.0
      (3, 6) :  None :    0.0 :   0.0
      (3, 7) :  None :    0.0 :   0.0
      (3, 8) :  None :    0.0 :   0.0
      (3, 9) :  None :    0.0 :   0.0
     (3, 10) :  None :    0.0 :   0.0
      (4, 1) :  None : -140.0 :   0.0
      (4, 2) :  None :    0.0 :   0.0
      (4, 3) :  None :    0.0 :   0.0
      (4, 4) :  None :    0.0 :   0.0
      (4, 5) :  None :    0.0 :   0.0
      (4, 6) :  None :    0.0 :   0.0
      (4, 7) :  None :    0.0 :   0.0
      (4, 8) :  None :    0.0 :   0.0
      (4, 9) :  None :    0.0 :   0.0
     (4, 10) :  None :    0.0 :   0.0
      (5, 1) :  None : -119.0 :   0.0
      (5, 2) :  None :    0.0 :   0.0
      (5, 3) :  None :    0.0 :   0.0
      (5, 4) :  None :    0.0 :   0.0
      (5, 5) :  None :    0.0 :   0.

In [65]:
Model._13.display()

_13 : Size=1
    Key  : Lower : Body : Upper
    None :  None :  0.0 :   0.0


In [66]:
Model._14.display()

_14 : Size=1
    Key  : Lower : Body : Upper
    None :  None :  0.0 :   0.0


In [67]:
Model._15.display()

_15 : Size=1
    Key  : Lower : Body : Upper
    None :   1.0 :  1.0 :   1.0


In [68]:
Model._16.display()

_16 : Size=1
    Key  : Lower : Body   : Upper
    None :  None : -134.0 :   0.0


In [69]:
Model._17.display()

_17 : Size=1
    Key  : Lower : Body   : Upper
    None :  None : -134.0 :   0.0


In [70]:
Model._18.display()

_18 : Size=1
    Key  : Lower : Body : Upper
    None :  None :  0.0 :   0.0


In [71]:
Model._19.display()

_19 : Size=1
    Key  : Lower : Body : Upper
    None :  None :  0.0 :   0.0


In [72]:
Model._20.display()

_20 : Size=1
    Key  : Lower : Body : Upper
    None :  None :  1.0 :   2.0


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 585
  Number of variables: 311
  Number of nonzeros: 1839
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.12800931930541992

Displaying Solution
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
objective : Size=1, Index=None, Acti

In [ ]:
from pyomo.environ import *


Model = ConcreteModel()

Model.n =  RangeSet(1,10)

Model.n0 = RangeSet(1,2)

Model.nc = RangeSet(3,10)






x1 = {1:117 , 2:79 , 3:92 , 4:20 ,5:83 , 6:69 , 7:42 , 8:94 , 9:74 , 10:69}

Model.x1 = Param(Model.n,initialize = x1)



y1 = {1:174 , 2:29 , 3:162 , 4:86 , 5:43 , 6:212 , 7 :50 , 8:118 , 9:64 , 10:194}

Model.y1 = Param(Model.n,initialize = y1)




d = {3:112 , 4:112 , 5:62 , 6:145 , 7:135 , 8:128 , 9:71 , 10:54}

Model.d = Param(Model.nc,initialize = d)



p = {3:23 , 4:200 , 5:13 , 6:200 , 7:27 , 8:200 , 9:15 , 10:98}

Model.p = Param(Model.nc,initialize = p)



fd ={1:36 , 2:33}

Model.fd = Param(Model.n0,initialize = fd)



cd = 1000


cv = 200

fv =20


c ={}
for i in Model.n:
    for j in Model.n:
               c[i,j] = ((x1[i]-x1[j])**2+ (y1[i]-y1[j])**2)**0.5

Model.c = Param(Model.n,Model.n ,initialize = c)



Model.x  = Var(Model.n , Model.n , within = Binary)


Model.y = Var(Model.n0  , within = Binary)

Model.z = Var(Model.nc , Model.n0  , within = Binary)


Model.v = Var(Model.n,Model.n , within = PositiveReals)

Model.u = Var(Model.n,Model.n , within = PositiveReals)




def _2 (Model,i):
     
        xij=0
        for j in Model.n:
            xij+=Model.x[i,j]
            return xij ==1
    
Model._2 = Constraint(Model.nc,rule =_2)



def _3 (Model,i):
    
        xij=0
        xji=0
        for j in Model.n:
            xij+=Model.x[i,j]
            xji+=Model.x[j,i]
        return xij == xji
Model._3 = Constraint(Model.n,rule =_3)



def _4 (Model,i):
    
        uij = 0
        uji = 0
        for j in Model.n:
            uji +=Model.u[j,i]
            uij +=Model.u[i,j]
        return uji - uij == Model.d[i]
Model._4 = Constraint(Model.nc,rule =_4)



def _5 (Model,i):
    
        vij = 0
        vji = 0
        for j in Model.n:
            vji +=Model.v[j,i]
            vij +=Model.v[i,j]
        return vji - vij == Model.p[i]
Model._5 = Constraint(Model.nc,rule =_5)


def _6 (Model,i,j):
  
            if i == j:
                Constraint.Skip
        
            return  Model.u[i,j] + Model.v[i,j] <=cv * Model.x[i,j]
            
                  
Model._6 = Constraint(Model.nc,Model.nc,rule =_6)



def _7 (Model,k):
    
        ukj = 0
        zjkdj = 0
        for j in Model.nc:
            ukj += Model.u[k,j]
            zjkdj +=Model.z[j,k]*Model.d[j]
        return ukj == zjkdj
Model._7 = Constraint(Model.n0,rule =_7)



def _8 (Model,k):
    
        ujk=0
        for j in Model.nc:
            ujk+=Model.u[j,k]
        return ujk == 0
Model._8 = Constraint(Model.n0,rule =_8)



def _9 (Model,k):
    
        vjk = 0
        zjkpj = 0
        for j in Model.nc:
            vjk += Model.v[k,j]
            zjkpj +=Model.z[j,k]*Model.p[j]
        return vjk == zjkpj
Model._9 = Constraint(Model.n0,rule =_9)



def _10 (Model,k):
    
        vkj=0
        for j in Model.nc:
            vkj+=Model.v[k,j]
        return vkj == 0
Model._10 = Constraint(Model.n0,rule =_10)




def _11 (Model,j,i):
   
            return Model.u[i,j] <= (cv - Model.d[i])* Model.x[i,j]
Model._11 = Constraint(Model.n,Model.nc,rule =_11)


def _12 (Model,i,j):
   
            return Model.v[i,j] <= (cv - Model.p[j])* Model.x[i,j]
Model._12 = Constraint(Model.n,Model.nc,rule =_12)



def _13 (Model,i,j):
   
            return Model.u[i,j] >= Model.d[j]*Model.x[i,j]
Model._13 = Constraint(Model.n,Model.nc,rule =_13)



def _14 (Model,j,i):
    
            return Model.v[i,j] >= Model.p[i] * Model.x[i,j]
Model._14 = Constraint(Model.n,Model.nc,rule =_14)

def _15 (Model,i):
    
        zik=0
        for k in Model.n0:
            zik += Model.z[i,k]
        return zik == 1
Model._15 = Constraint(Model.nc,rule =_15)

def _16 (Model,k):
    
        dizik = 0
        for i in Model.nc:
            dizik += Model.d[i] * Model.z[i,k]
        return dizik <=cd * Model.y[k]
Model._16 = Constraint(Model.n0,rule =_16)

def _17 (Model,k):
    
        pizik = 0
        for i in Model.nc:
            pizik += Model.p[i] * Model.z[i,k]
        return pizik <=cd * Model.y[k]
Model._17 = Constraint(Model.n0,rule =_17)

def _18 (Model,k):
    
        for i in Model.nc:
            return Model.x[i,k] <= Model.z[i,k]
Model._18 = Constraint(Model.n0,rule =_18)

def _19 (Model,k,i):
   
            return Model.x[k,i] <= Model.z[i,k]
Model._19 = Constraint(Model.n0,Model.nc,rule =_19)

def _20 (Model,i,j,k):
    
                if i==j :
                    Constraint.Skip
                zjm = 0
                for m in Model.n0:
                    if m == k:
                            Constraint.Skip
                    
                    zjm += Model.z[j,m]
                    return Model.x[i,j]+Model.z[i,k] + zjm <=2
Model._20 = Constraint(Model.nc,Model.nc,Model.n0,rule =_20)


def obj (Model):
    cx = 0
    fdy = 0
    fvx = 0
    for i in Model.n :
        for j in Model.n :
            cx += Model.c[i,j]*Model.x[i,j]
    for k in Model.n0 :
        fdy += Model.fd[k] * Model.y[k]
    for k in Model.n0 :
        for i in Model.nc :
            fvx += fv * Model.x[k,i]
    return cx + fdy + fvx
Model.objective = Objective(rule=obj)



def pyomo_postprocess(options=None, instance=None, results=None):
        Model.objective.display()
        

    #This is an optional code path that allows the script to be run outside of
    #pyomo command-line.  For example:  python transport.py
if __name__ == '__main__':
   # This emulates what the pyomo command-line tools does
    from pyomo.opt import SolverFactory
#   import pyomo.environ
    opt = SolverFactory("glpk")
    results = opt.solve(Model)
    #sends results to stdout
#    results.write()
#    print("\nDisplaying Solution\n" + '!'*60)
     pyomo_postprocess(None, Model, results)